In [28]:
from os import listdir
import pandas
import re

from pg import IntegrityError

from pg import DB
db = DB()

### Creating database table to store sample scores

In [6]:
db.query("""create table samples_scores_mini(
         size integer,
         average_score float,
         tt_average_score float
        )""")

### Inserting samples scores into database

In [8]:
%%time
for sample_file_name in sample_file_names:
    print(sample_file_name)
    size = int(re.search(r'\d+', sample_file_name).group())
    sample = pandas.read_pickle(open( "../files/intermediates/samples/"+sample_file_name, "rb" ))
    sample_df =pandas.DataFrame(sample)
    pandas.DataFrame(sample)
    for i, row in enumerate(sample_df["average_scores"]):
        do_not_print = db.insert('samples_scores',
                             size = size,
                             average_score = sample_df["average_scores"][i],
                             tt_average_score = sample_df["tt_average_scores"][i]
                            )

sample_size:10_samples:1000.pkl
CPU times: user 5.26 ms, sys: 2.26 ms, total: 7.52 ms
Wall time: 143 ms


### Creating database table to store motif scores

In [10]:
db.query("""create table motif_average_scores_mini(
         motif_string varchar primary key,
         proteins_length integer,
         average_score float,
         tt_average_score float,
         shuffled_proteins_length integer,
         shuffled_average_score float,
         tt_shuffled_average_score float
        )""")

### Inserting motif scores into database

In [58]:
%%time
motif_scores_file_names = listdir("../files/motif_scores/")

try:
    for motif_scores_file_name in motif_scores_file_names:
        print(motif_scores_file_name)
        motif_scores = pandas.read_pickle(open( "../files/motif_scores/"+motif_scores_file_name, "rb" ))
        motif_scores_df =pandas.DataFrame(motif_scores)
    #     pandas.DataFrame(sample)
        for i, row in enumerate(motif_scores_df["average_scores"]):
    #         print(motif_scores_df.index[i], motif_scores_df["motif_subgraph_size"][i], motif_scores_df["average_scores"][i])
            do_not_print = db.insert('motif_average_scores_mini',
                                 motif_string = motif_scores_df.index[i],
                                 proteins_length = motif_scores_df["motif_subgraph_size"][i],
                                 average_score = motif_scores_df["average_scores"][i],
                                 tt_average_score = motif_scores_df["average_tt_scores"][i],
                                 shuffled_proteins_length = motif_scores_df["motif_shuffled_subgraph_size"][i],
                                 shuffled_average_score = motif_scores_df["shuffled_average_scores"][i],
                                 tt_shuffled_average_score = motif_scores_df["shuffled_average_tt_scores"][i]
                                )
except IntegrityError as e:
    print(e)


numJobs:100000_jobNumber:10.pkl
CPU times: user 14.1 ms, sys: 2.68 ms, total: 16.8 ms
Wall time: 109 ms


### Adding collumns for p_values into the motifs scores table

In [33]:
db.query("ALTER TABLE motif_average_scores_mini ADD COLUMN p_value float;")
db.query("ALTER TABLE motif_average_scores_mini ADD COLUMN shuffled_p_value float;")
db.query("ALTER TABLE motif_average_scores_mini ADD COLUMN tt_p_value float;")
db.query("ALTER TABLE motif_average_scores_mini ADD COLUMN tt_shuffled_p_value float;")

### Updating p-values for motifs

In [54]:
%%time
p_values_file_names = listdir("../files/intermediates/p_values/")
for p_values_file in p_values_file_names:
    p_values = pandas.read_pickle(open( "../files/intermediates/p_values/"+p_values_file, "rb" ))
    p_values_df =pandas.DataFrame(p_values)
    print("dod")
    for i, row in enumerate(p_values_df.index):
        do_not_print = db.query("UPDATE motif_average_scores_mini SET p_value = '" + str(p_values_df['motif_p_value'][row]) + "', shuffled_p_value = '" + str(p_values_df['motif_shuffled_p_value'][row]) + "', tt_p_value = '" + str(p_values_df['motif_tt_p_value'][row]) + "', tt_shuffled_p_value = '" +str(p_values_df['motif_shuffled_tt_p_value'][row]) +  "' WHERE motif_string  = '" + str(row) + "'")



dod
CPU times: user 1.13 ms, sys: 763 µs, total: 1.9 ms
Wall time: 1.32 ms


Peeking into p_values. 

NaN values represent entries where the p-value was not calculated because the score was not calculated since the subgraph was bigger than our cut off size. If both the subgraph and subgraph of the shuffled results are bigger than cutoff value the motif is not even in the table below.

In [55]:
pandas.DataFrame(p_values)

,motif_p_value,motif_shuffled_p_value,motif_shuffled_tt_p_value,motif_tt_p_value


In [56]:
p_values

,motif_p_value,motif_shuffled_p_value,motif_shuffled_tt_p_value,motif_tt_p_value


In [57]:
motif_scores

,average_scores,average_tt_scores,motif_shuffled_subgraph_size,motif_subgraph_size,shuffled_average_scores,shuffled_average_tt_scores
aaaac.aa,NaN,NaN,953,1009,NaN,NaN
aaaac[ag]..,NaN,NaN,2079,2055,NaN,NaN
aaaac[ag].[ag],NaN,NaN,1566,1556,NaN,NaN
aaaac[ag].[ct],NaN,NaN,1381,1396,NaN,NaN
aaaac[ag].a,NaN,NaN,1230,1210,NaN,NaN
aaaac[ag].c,NaN,NaN,738,697,NaN,NaN
aaaac[ag].g,NaN,NaN,762,732,NaN,NaN
aaaac[ag].t,NaN,NaN,975,1035,NaN,NaN
aaaac[ag][ct].,NaN,NaN,1372,1370,NaN,NaN
aaaac[ct]..,NaN,NaN,1872,2189,NaN,NaN
